In [43]:
import os
import csv
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
# from sentence_transformers import SentenceTransformer, util
# import networkx as nx
# from pyvis.network import Network

# read data

In [59]:
all_posts_df = pd.read_csv('../data/merged_q/all_posts_max_len_10.csv')
all_posts_df.head()
print(all_posts_df.dtypes)

id            object
summarized     int64
text          object
root          object
length         int64
dtype: object


In [3]:
all_replies_df = pd.read_csv('../data/merged_q/all_replies_max_len_10.csv')
all_replies_df.head()

,id,parent_id,summarized,text,length
0,hm5iqwj,r2nmvy,1,Conscious Club is campy in a good way,8
1,hm5kaqv,r2nmvy,0,I like Scary Pockets.,5
2,hm5kddu,r2nylj,0,what keeps me calm is my prescription medication,8
3,hm5kig9,r2nylj,1,Wow you're amazingly talented.,6
4,hm5kl30,r2npa3,0,Walking in the countryside withan audiobook.,7


# load model

In [40]:
import sys
sys.setrecursionlimit(1000000)

In [45]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU



# RAM

# GPU

# OS
Darwin MagicMac 20.6.0 Darwin Kernel Version 20.6.0: Tue Oct 12 18:33:42 PDT 2021; root:xnu-7195.141.8~1/RELEASE_X86_64 x86_64


cat: /proc/cpuinfo: No such file or directory
cat: /proc/cpuinfo: No such file or directory
cat: /proc/cpuinfo: No such file or directory
cat: /proc/meminfo: No such file or directory
/bin/sh: lspci: command not found


In [2]:
import tensorflow as tf

model = tf.keras.models.load_model("../Colbert/colbert-trained/")
model.summary()

2022-03-16 21:54:56.677612: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

                                                                 ]']                              
                                                                                                  
 dense_21 (Dense)               (None, 32)           24608       ['global_average_pooling1d_9[0][0
                                                                 ]']                              
                                                                                                  
 dense_23 (Dense)               (None, 32)           24608       ['global_average_pooling1d_10[0][
                                                                 0]']                             
                                                                                                  
 dense_25 (Dense)               (None, 256)          196864      ['global_average_pooling1d_11[0][
                                                                 0]']                             
          

In [42]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
# import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

RuntimeError: Failed to import transformers.models.fnet.tokenization_fnet because of the following error (look up to see its traceback):
No module named 'sentencepiece'

# 2. Preprocessing functions


In [8]:
from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

/Users/johnchen/opt/anaconda3/envs/SP/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|████████████████████████████| 226k/226k [00:00<00:00, 588kB/s]
Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 9.13kB/s]
Downloading: 100%|██████████████████████████████| 570/570 [00:00<00:00, 243kB/s]


In [46]:
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [10]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input

In [11]:
training_sample_count = 1000 # 4000
test_count = 1000

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [12]:
all_posts_df_test=all_posts_df.sample(n=10,random_state=1)

In [13]:
all_posts_df_test

,id,summarized,text,root,length
5166,lmfqjq,0,I can see Mount Fuji outside my window,see,8
47982,7or70u,0,Im at home waiting for a robber,be,8
3447,nsp4wk,1,I look back and not one thing went wrong.,look,10
34177,a713ot,1,Would love to chat for a distraction!,love,8
50478,7b26kv,1,Why even put THAT much effort into a reply?,put,10
39368,94rroc,1,I started university last year around October.,start,8
51131,77r5f1,0,What’s keeping you up tonight?,keep,7
15174,fxo0ry,0,Trying to stay up cause my cat is sick,try,9
25169,d1xlnh,0,So I forgot my cake day,forget,6
20567,ekxmb5,1,My ex boyfriend's fuck buddy bought me presents.,buy,10


In [53]:
import datetime
starttime = datetime.datetime.now()

# input_categories = list(all_posts_df_test.columns[[2]])
input_categories=['text']
test_inputs = compute_input_arrays(all_posts_df_test, input_categories, tokenizer)
all_posts_df_test['humor']=model.predict(test_inputs)
all_posts_df_test['humor'] = all_posts_df_test['humor'].apply(lambda x:True if x>0.7 else False)


endtime = datetime.datetime.now()
print (endtime - starttime)





KeyError: "None of [Index(['text'], dtype='object')] are in the [columns]"

In [54]:
import datetime
starttime = datetime.datetime.now()

colbert_df = pd.read_csv('../data/matched_q/casual_conv_20160101_20161231.csv')
# input_categories = list(all_posts_df_test.columns[[2]])
input_categories=['src_text']
test_inputs = compute_input_arrays(colbert_df, input_categories, tokenizer)
colbert_df['humor']=model.predict(test_inputs)
colbert_df['humor'] = all_posts_df_test['humor'].apply(lambda x:True if x>0.7 else False)


endtime = datetime.datetime.now()

0it [00:00, ?it/s]


AttributeError: 'Series' object has no attribute 'text'

In [57]:

colbert_df = pd.read_csv('../data/matched_q/casual_conv_20160101_20161231.csv')
colbert_df=colbert_df.sample(n=10,random_state=1)
colbert_df['combined'] =colbert_df['src_text']+' '+colbert_df['com_text']
input_categories=['combined']
# print(colbert_df)

print(colbert_df.dtypes)

# test_inputs = compute_input_arrays(colbert_df, input_categories, tokenizer)
# for i in tqdm(range(colbert_df.shape[0])):
#     colbert_df.iloc[i]['humor']=model.predict(test_inputs[i])

src_id             object
src_type           object
src_summarized      int64
src_from           object
src_text           object
src_root           object
src_length          int64
com_id             object
com_summarized      int64
com_text           object
com_length          int64
com_score         float64
combined           object
dtype: object


In [49]:
colbert_df

,src_id,src_type,src_summarized,src_from,src_text,src_root,src_length,com_id,com_summarized,com_text,com_length,com_score,combined
1743,dfbm0ii,com,0,comment,*cricris*,NaN,3,dfbra18,0,I WIN,2,NaN,*cricris* I WIN
969,dcg0p50,com,0,comment,Me too Tights,NaN,3,dcg14yr,0,Cat Tights,2,NaN,Me too Tights Cat Tights
1355,dd1h0nk,com,0,comment,ALL.,NaN,2,dd1h5xd,1,MISTAKEN.,2,NaN,ALL. MISTAKEN.
2159,dd5f1tz,com,0,comment,"Oh, silly Mozilla...",NaN,5,dd5o05g,0,but it was so boring 3xx,6,NaN,"Oh, silly Mozilla... but it was so boring 3xx"
1897,dcchr6y,com,0,comment,Any bright ideas :I,NaN,5,dcci12p,0,Lamps.,2,NaN,Any bright ideas :I Lamps.
1283,dckdc2c,com,0,comment,DAS NOT TROOOOOO,NaN,3,dckdpyt,0,PUH ROOF ITTTTTTT,3,NaN,DAS NOT TROOOOOO PUH ROOF ITTTTTTT
1009,dcko6if,com,0,comment,Military Tights,NaN,2,dckoa9k,0,Barracks Tights,2,NaN,Military Tights Barracks Tights
982,dcg2dc8,com,0,comment,Stocking Tights,NaN,2,dcg2dwq,0,Garter Tights,2,NaN,Stocking Tights Garter Tights
1213,dc92bpt,com,0,comment,Ohh goodness.,NaN,3,dc968cv,1,Yeahh.,2,NaN,Ohh goodness. Yeahh.
484,dc0lvfe,com,0,comment,Ebay Tights,NaN,2,dc0m02i,0,Craigslist Tights,2,NaN,Ebay Tights Craigslist Tights


In [35]:
date_suffices = [
#     '20210501_20211231', '20210101_20210430', '20200101_20201231',
#                  '20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231'
                ]
colbert_df = pd.read_csv('../data/matched_q/casual_conv_20160101_20161231.csv')
colbert_df
colbert_df['combined'] = colbert_df['src_text']+' '+colbert_df['com_text']



,src_id,src_type,src_summarized,src_from,src_text,src_root,src_length,com_id,com_summarized,com_text,com_length,com_score,combined
5,5lce7x,sub,0,title,How are you celebrating your news year eve?,celebrate,9,dbumrm1,1,HOORAY!,2,NaN,How are you celebrating your news year eve? HO...
6,5lce7x,sub,0,title,How are you celebrating your news year eve?,celebrate,9,dbun00g,1,Watched the countdown on TV now I'm gonna read...,20,NaN,How are you celebrating your news year eve? Wa...
7,5lce7x,sub,0,title,How are you celebrating your news year eve?,celebrate,9,dbun0y4,1,Hope that you're having a nice time too.,10,NaN,How are you celebrating your news year eve? Ho...
8,5lce7x,sub,0,title,How are you celebrating your news year eve?,celebrate,9,dbundan,1,Happy new year!,4,NaN,How are you celebrating your news year eve? Ha...
9,5lce7x,sub,0,title,How are you celebrating your news year eve?,celebrate,9,dbuo77q,1,Going on a yacht in marina del ray.,9,NaN,How are you celebrating your news year eve? Go...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,dgfkiax,com,0,comment,Being easily annoyed translates to being mean?,NaN,8,dghbmdj,0,Uh huh.,3,NaN,Being easily annoyed translates to being mean?...
2118,dcexeyz,com,0,comment,So how are you?,NaN,5,dcg7pw5,1,Getting food tomorrow.,4,NaN,So how are you? Getting food tomorrow.
2126,dcnyjeb,com,0,comment,Oh yeah?,NaN,3,dd2ekl1,0,More shit happened!!!,6,NaN,Oh yeah? More shit happened!!!
2131,dcp86v8,com,0,comment,Why?,NaN,2,dcp8d22,0,I dunno she just is,5,NaN,Why? I dunno she just is


In [ ]:
date_suffices = [
#     '20210501_20211231', '20210101_20210430', '20200101_20201231',
#                  '20190101_20191231', '20180101_20181231', '20170101_20171231',
                 '20160101_20161231'
                ]
for date_suffix in date_suffices:
    colbert_df = pd.read_csv('../data/matched_q/casual_conv_{}.csv'.format(date_suffix))
    colbert_df['combined'] = colbert_df['src_text']+colbert_df['com_text']



In [24]:
all_posts_df_test

,id,summarized,text,root,length,humor
5166,lmfqjq,0,I can see Mount Fuji outside my window,see,8,0.992325
47982,7or70u,0,Im at home waiting for a robber,be,8,0.999689
3447,nsp4wk,1,I look back and not one thing went wrong.,look,10,0.999849
34177,a713ot,1,Would love to chat for a distraction!,love,8,0.707652
50478,7b26kv,1,Why even put THAT much effort into a reply?,put,10,0.065793
39368,94rroc,1,I started university last year around October.,start,8,0.999950
51131,77r5f1,0,What’s keeping you up tonight?,keep,7,0.058982
15174,fxo0ry,0,Trying to stay up cause my cat is sick,try,9,0.981470
25169,d1xlnh,0,So I forgot my cake day,forget,6,0.878770
20567,ekxmb5,1,My ex boyfriend's fuck buddy bought me presents.,buy,10,0.999948


In [ ]:
pickle_path='../data/pickle/all_posts_df.pkl'
all_posts_df.to_pickle(out_file)

# predict

In [18]:
all_posts_df


,id,summarized,text,root,length
0,r2vbm3,0,Does pressing keyboard keys reflect your perso...,reflect,8
1,r2unhy,0,Need some socialization this week ☺️,need,7
2,r2tj6o,1,Quarter life crises starts now maybe haha?,start,8
3,r2sz0f,1,Do I want to get better?,want,7
4,r2sj5n,0,Should I drop them?,drop,5
...,...,...,...,...,...
65012,hit:12249_conv:24498_uttr:1,1,I love life these days!,love,6
65013,hit:12307_conv:24614_uttr:1,1,I recently tried taking an ice bath,try,7
65014,hit:12384_conv:24768_uttr:1,0,"I almost got caught stealing, I felt pretty bad",feel,10
65015,hit:12398_conv:24797_uttr:1,1,I saw my favourite candy at the store today!,see,10


In [ ]:
preds = model.predict(valid_inputs)